In [15]:
import time
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests as rq
import re
from time import sleep
from random import randint
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from IPython.core.display import clear_output

#Chinatown : https://www.burpple.com/search/sg?utf8=%E2%9C%93&sort=trending&category_id=5&area_id=18&distance_to=0.0&price_from=0&price_to=&q=chinatown+cafe&ll=&commit=Apply
#Marina Bay : https://www.burpple.com/search/sg?utf8=%E2%9C%93&sort=trending&category_id=5&area_id=25&distance_to=0.0&price_from=0&price_to=&q=marina+bay+cafe&ll=&commit=Apply
#Telok Ayer : https://www.burpple.com/search/sg?utf8=%E2%9C%93&sort=trending&category_id=5&area_id=140&distance_to=0.0&price_from=0&price_to=&q=telok+ayer+cafe&ll=&commit=Apply
#Tanjong Pagar : https://www.burpple.com/search/sg?utf8=%E2%9C%93&sort=trending&category_id=5&area_id=&distance_to=0.0&price_from=0&price_to=&q=tanjong+pagar+cafe&ll=&commit=Apply

page = rq.get("https://www.burpple.com/search/sg?utf8=%E2%9C%93&sort=trending&category_id=5&area_id=140&distance_to=0.0&price_from=0&price_to=&q=telok+ayer+cafe&ll=&commit=Apply")
#download the website, to know if succcessful, write 'page' then run
#the status should come out 200

# I used Chrome; you can use whichever browser you like.
driver = webdriver.Chrome()

# Tell Selenium to get the URL you're interested in.
driver.get("https://www.burpple.com/search/sg?utf8=%E2%9C%93&sort=trending&category_id=5&area_id=140&distance_to=0.0&price_from=0&price_to=&q=telok+ayer+cafe&ll=&commit=Apply")


# Selenium script to scroll to the bottom, wait a few seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
                lastCount = lenOfPage
                time.sleep(6)
                lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
                time.sleep(8)
                if lastCount==lenOfPage:
                    match=True

# Now that the page is fully scrolled, grab the source code.
source_data = driver.page_source

# Throw your source into BeautifulSoup and start parsing!
soup = bs(source_data,'lxml')

container = soup.find_all("div",{"class": "searchVenue-header-left"})

names = []
neigh = []
review = []
price = []
long = []
lat = []


y = len(container)

for item in container:
    
    review_elem = item.find("span",{"class": "searchVenue-header-reviews"}).text
    
#you can only get .text from find()
#if you want to get from findall() you need to specify the index eg. [0]

   #must put .text if not the review_elem will be the html code instead of specific text you want
    reviewstrip = review_elem.strip('Reviews": ')
    intreview = int(reviewstrip)
        
    if (intreview > 4 ):
    #in case some cafes dont have this information
  
        
        names_elem = item.find("span",{"class": "searchVenue-header-name-name headingMedium"}).text
        neigh_elem = item.find("span",{"class": "searchVenue-header-locationDistancePrice-location"}).text
        long_elem = item.find("span",{"class": "searchVenue-header-locationDistancePrice-distance"}).get('data-longitude')
        lat_elem = item.find("span",{"class": "searchVenue-header-locationDistancePrice-distance"}).get('data-latitude')
        floatlong = float(long_elem)
        floatlat = float(lat_elem)
        
        
            
        try:
            price_elem = item.find("span",{"class": "searchVenue-header-locationDistancePrice-price"}).text
            price_digit = price_elem.replace(price_elem[:5], '')
            price_strip = price_digit.strip('/pax": ')
            intprice = int(price_strip)
        #i only want the digits
        except AttributeError:
            intprice = '-'
    
        price.append(intprice)
        names.append(names_elem)
        neigh.append(neigh_elem)
        review.append(intreview)
        long.append(floatlong)
        lat.append(floatlat)
   

In [16]:
BurppleIndividualCafe = pd.DataFrame({
        "Name_Of_Cafe" : names,
        "Neighbourhood" : neigh,
        "Latitude" : lat,
        "Longitude" : long, 
        "No_Of_Reviews" : review,
        "Price_per_pax" : price})

In [17]:
##Reorder the sequence 
BurppleIndividualCafe = BurppleIndividualCafe[['Name_Of_Cafe', 'Neighbourhood', 'Latitude','Longitude','No_Of_Reviews', 'Price_per_pax']]
BurppleIndividualCafe

,Name_Of_Cafe,Neighbourhood,Latitude,Longitude,No_Of_Reviews,Price_per_pax
0,Maxi Coffee Bar,Chinatown,1.282349,103.846380,15,10
1,Free The Robot Coffee,Telok Ayer,1.281972,103.848170,219,20
2,Grain Traders (Telok Ayer),Raffles Place,1.281533,103.849938,166,20
3,Pantler,Tanjong Pagar,1.280198,103.847261,156,15
4,Sarnies (Telok Ayer),Telok Ayer,1.281613,103.848004,407,20
5,Chong Wen Ge Cafe 崇文阁,Tanjong Pagar,1.280785,103.847591,88,10
6,Birds of a Feather,Amoy Street,1.282141,103.847711,242,50
7,Kitchen by Food Rebel,Tanjong Pagar,1.280880,103.848305,16,20
8,Common Man Stan,Tanjong Pagar,1.280279,103.847971,36,30
9,PS.Cafe (Ann Siang Hill),Tanjong Pagar,1.280391,103.846217,182,40


In [18]:
ex = pd.ExcelWriter('CompiledIndividualCafes.xlsx', options={'encoding':'utf-8'})
BurppleIndividualCafe.to_excel(ex, 'Data', index=False)
ex.save()


In [19]:
  
    
   ## I have another script solely for cafe links, but since we are using the same page source we can just add
   # part of the extracting codes in here to save time.


container2 = soup.find_all("div",{"class": "searchVenue-header card-item card-item--header"})
    

links = []
names2 = []

for item2 in container2:
        
    review_elem2 = item2.find("span",{"class": "searchVenue-header-reviews"}).text
    
#you can only get .text from find()
#if you want to get from findall() you need to specify the index eg. [0]

   #must put .text if not the review_elem will be the html code instead of specific text you want
    reviewstrip2 = review_elem2.strip('Reviews": ')
    intreview2 = int(reviewstrip2)
        
        
    if (intreview2 > 4 ):
    #in case some cafes dont have this information
        link_elem = item2.find('a', attrs={'href': re.compile("^/")})
    #re.compile("^/") is to match the url starting with /
        links.append(link_elem.get('href'))
        
        names_elem2 = item.find("span",{"class": "searchVenue-header-name-name headingMedium"}).text
        names2.append(names_elem2)


In [20]:
testlink = pd.DataFrame({
        "Cafe_Name ": names,
        "Links" : links
    
    })

testlink


,Cafe_Name,Links
0,Maxi Coffee Bar,/maxi-coffee-bar?bp_ref=%2Fsearch%2Fsg
1,Free The Robot Coffee,/free-the-robot-coffee?bp_ref=%2Fsearch%2Fsg
2,Grain Traders (Telok Ayer),/grain-traders?bp_ref=%2Fsearch%2Fsg
3,Pantler,/pantler?bp_ref=%2Fsearch%2Fsg
4,Sarnies (Telok Ayer),/sarnies?bp_ref=%2Fsearch%2Fsg
5,Chong Wen Ge Cafe 崇文阁,/chong-wen-ge-cafe?bp_ref=%2Fsearch%2Fsg
6,Birds of a Feather,/birds-of-a-feather-sg?bp_ref=%2Fsearch%2Fsg
7,Kitchen by Food Rebel,/kitchen-by-food-rebel?bp_ref=%2Fsearch%2Fsg
8,Common Man Stan,/common-man-stan?bp_ref=%2Fsearch%2Fsg
9,PS.Cafe (Ann Siang Hill),/pscafe-at-ann-siang-hill-park?bp_ref=%2Fsearc...


In [21]:
ex = pd.ExcelWriter('CompiledCafeLinks.xlsx', options={'encoding':'utf-8'})
testlink.to_excel(ex, 'Data', index=False)
ex.save()

In [ ]:
##Save time for subsequent scripts, so dont need parse the burpple raffles cafes again to find the links

import pandas as pd

xl_workbook = pd.ExcelFile('C:/Users/Mei Xuan/Downloads/CompiledCafeLinks.xlsx')  # Load the excel workbook
df = xl_workbook.parse("Data")  # Parse the sheet into a dataframe
aList = df['Links'].tolist()

##Test if the link will work
print("Test Next Page URL: {}".format('https://www.burpple.com'+aList[0]))